In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [3]:
raw_documents = WikipediaLoader(query="Atelectasis").load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents([raw_documents[0]]) # should I only take the first? which is the most relevant?

In [13]:
len(raw_documents)

10

In [4]:
from tqdm import tqdm
from aug.graph_doc import (
    get_extraction_chain,
    data_to_graph_doc,
    chain_run,
    add_graph_documents,
)
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

allowed_nodes = None
allowed_rels = None
# allowed_nodes = ["Symptom", "Disease"]
# allowed_rels = ["CAN_CAUSE", "DESCRIBE", "HAS"]

extract_chain = get_extraction_chain(llm, allowed_nodes, allowed_rels)
gds = []

for d in tqdm(documents, total=len(documents)):
    data = chain_run(extract_chain, d.page_content)
    # data = extract_chain.run(d.page_content)
    # graph_document = GraphDocument(
    #     nodes=[map_to_base_node(node) for node in data.nodes],
    #     relationships=[map_to_base_relationship(rel) for rel in data.rels],
    #     source=d,
    # )
    graph_document = data_to_graph_doc(data, d)
    # add_graph_document(graph, graph_document)
    gds.append(graph_document)

add_graph_documents(graph, gds)

graph.refresh_schema()
print(graph.schema)

100%|██████████| 1/1 [00:38<00:00, 38.22s/it]



        Node properties are the following:
        [{'labels': 'Condition', 'properties': [{'property': 'causedBy', 'type': 'STRING'}, {'property': 'affects', 'type': 'STRING'}, {'property': 'description', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}, {'property': 'name', 'type': 'STRING'}, {'property': 'definition', 'type': 'STRING'}, {'property': 'unilateral', 'type': 'STRING'}, {'property': 'alsoKnownAs', 'type': 'STRING'}]}, {'labels': 'Section', 'properties': [{'property': 'name', 'type': 'STRING'}, {'property': 'id', 'type': 'STRING'}]}, {'labels': 'Symptom', 'properties': [{'property': 'id', 'type': 'STRING'}]}, {'labels': 'Cause', 'properties': [{'property': 'id', 'type': 'STRING'}]}, {'labels': 'Risk factor', 'properties': [{'property': 'id', 'type': 'STRING'}]}, {'labels': 'Subsection', 'properties': [{'property': 'id', 'type': 'STRING'}]}, {'labels': 'Diagnostic tool', 'properties': [{'property': 'id', 'type': 'STRING'}]}, {'labels': 'Diagnostic finding', 'proper

In [5]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    validate_cypher=True,  # Validate relationship directions
    verbose=True,
)

In [6]:
chain.run("What is Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})
RETURN c
Full Context:
[{'c': {'name': 'Atelectasis', 'causedBy': 'normal exhalation or various medical conditions', 'affects': 'part or all of one lung', 'description': 'A condition where the alveoli are deflated down to little or no volume, as distinct from pulmonary consolidation, in which they are filled with liquid', 'definition': 'The collapse or closure of a lung resulting in reduced or absent gas exchange', 'unilateral': 'true', 'id': 'Atelectasis', 'alsoKnownAs': 'collapsed lung'}}]

> Finished chain.


'Atelectasis is a condition where part or all of one lung collapses or closes, resulting in reduced or absent gas exchange. It can be caused by normal exhalation or various medical conditions. The alveoli, which are small air sacs in the lungs, become deflated down to little or no volume. Atelectasis is also known as a collapsed lung.'

In [7]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})-[:HAS]->(s:Section)-[:INCLUDES]->(sym:Symptom)
RETURN sym.name
Full Context:
[{'sym.name': None}, {'sym.name': None}, {'sym.name': None}, {'sym.name': None}, {'sym.name': None}, {'sym.name': None}, {'sym.name': None}]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question about the symptoms of Atelectasis."

In [8]:
chain.run("What can cause Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})-[:HAS]->(s:Section)-[:INCLUDES]->(cause:Cause)
RETURN cause.name
Full Context:
[{'cause.name': None}, {'cause.name': None}, {'cause.name': None}, {'cause.name': None}, {'cause.name': None}, {'cause.name': None}]

> Finished chain.


"I'm sorry, but I don't have any information on what can cause Atelectasis."

In [9]:
chain.run("What clinical features radiologists need for diagnosing Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})-[:HAS]->(s:Section)-[:INCLUDES]->(sym:Symptom)
RETURN sym.name AS ClinicalFeaturesRadiologistsNeed
Full Context:
[{'ClinicalFeaturesRadiologistsNeed': None}, {'ClinicalFeaturesRadiologistsNeed': None}, {'ClinicalFeaturesRadiologistsNeed': None}, {'ClinicalFeaturesRadiologistsNeed': None}, {'ClinicalFeaturesRadiologistsNeed': None}, {'ClinicalFeaturesRadiologistsNeed': None}, {'ClinicalFeaturesRadiologistsNeed': None}]

> Finished chain.


"I'm sorry, but I don't have the information on the clinical features that radiologists need for diagnosing Atelectasis."

In [11]:
chain.run("what are radiographic features of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Condition {name: 'Atelectasis'})-[:HAS]->(s:Section)-[:INCLUDES]->(df:Diagnostic finding)
RETURN df.name


ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'finding': expected ")", "WHERE", "{" or a parameter (line 1, column 91 (offset: 90))
"MATCH (c:Condition {name: 'Atelectasis'})-[:HAS]->(s:Section)-[:INCLUDES]->(df:Diagnostic finding)"
                                                                                           ^}